In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  
os.environ["CUDA_VISIBLE_DEVICES"]="0,7" 

In [2]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
from torch.autograd import Variable

from lib.utils.config_parse import cfg_from_file
from lib.ssds_train import test_model, train_model

from lib.dataset.dataset_factory import load_data
from lib.utils.config_parse import cfg
from lib.modeling.model_builder import create_model

from lib.layers import Detect
from lib.utils.timer import Timer

import sys
import pickle
import numpy as np



config_file =  './experiments/cfgs/ssd_resnet50_train_voc.yml'
cfg_from_file(config_file)

print(config_file)

./experiments/cfgs/ssd_resnet50_train_voc.yml


In [3]:
train_model()

===> Loading data
===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]
Utilize GPUs for computation
Number of GPU available 2
Model architectures:
SSD(
  (base): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): _bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, mome

/workspace/home/jgusak/ssds.pytorch/lib/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)
/opt/conda/lib/python3.6/site-packages/torch/nn/_reduction.py:46: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


IndexError: The shape of the mask [32, 11620] at index 0 does not match the shape of the indexed tensor [371840, 1] at index 0

In [3]:
 # Load data
print('===> Loading data')
train_loader = load_data(cfg.DATASET, 'train') if 'train' in cfg.PHASE else None
eval_loader = load_data(cfg.DATASET, 'eval') if 'eval' in cfg.PHASE else None
test_loader = load_data(cfg.DATASET, 'test') if 'test' in cfg.PHASE else None

===> Loading data


In [4]:
# Build model
print('===> Building model')
model, priorbox = create_model(cfg.MODEL)
with torch.no_grad():
    priors = Variable(priorbox.forward())
    detector = Detect(cfg.POST_PROCESS, priors)

===> Building model
==>Feature map size:
[(38, 38), (19, 19), (10, 10), (5, 5), (3, 3), (1, 1)]


/workspace/home/jgusak/ssds.pytorch/lib/layers/modules/l2norm.py:17: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(self.weight,self.gamma)


In [5]:
# Load pretrained weights

state_dict_path = "/workspace/raid/data/jgusak/ssds.pytorch/resnet50_ssd_voc_79.7.pth"
state_dict = torch.load(state_dict_path)


model_dict = model.state_dict()
pretrained_dict = {k: v for k, v in state_dict.items() 
                   if k in model_dict}

model_dict.update(pretrained_dict)
model.load_state_dict(model_dict)
# model.load_state_dict(torch.load(state_dict_path))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [6]:
# Utilize GPUs for computation
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('Utilize GPUs for computation')
    print('Number of GPU available', torch.cuda.device_count())
    model.cuda()
    priors.cuda()
    cudnn.benchmark = True

Utilize GPUs for computation
Number of GPU available 2


In [7]:
output_dir = cfg.EXP_DIR
checkpoint = cfg.RESUME_CHECKPOINT
checkpoint_prefix = cfg.CHECKPOINTS_PREFIX

checkpoint, checkpoint_prefix

('/workspace/raid/data/jgusak/ssds.pytorch/resnet50_ssd_voc_79.7.pth',
 'ssd_resnet_50_voc')

In [8]:
def test_epoch(model, data_loader, detector, output_dir, use_gpu):
        model.eval()

        dataset = data_loader.dataset
        num_images = len(dataset)
        num_classes = detector.num_classes
        all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]
        empty_array = np.transpose(np.array([[],[],[],[],[]]),(1,0))

        _t = Timer()

        with torch.no_grad():
            for i in iter(range((num_images))):
                img = dataset.pull_image(i)
                scale = [img.shape[1], img.shape[0], img.shape[1], img.shape[0]]
                if use_gpu:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0).cuda())
                else:
                    images = Variable(dataset.preproc(img)[0].unsqueeze(0))
                _t.tic()
                # forward
                out = model(images, phase='eval')

                # detect
                detections = detector.forward(out)

                time = _t.toc()

                # TODO: make it smart:
                for j in range(1, num_classes):
                    cls_dets = list()
                    for det in detections[0][j]:
                        if det[0] > 0:
                            d = det.cpu().numpy()
                            score, box = d[0], d[1:]
                            box *= scale
                            box = np.append(box, score)
                            cls_dets.append(box)
                    if len(cls_dets) == 0:
                        cls_dets = empty_array
                    all_boxes[j][i] = np.array(cls_dets)

                # log per iter
                log = '\r==>Test: || {iters:d}/{epoch_size:d} in {time:.3f}s [{prograss}]\r'.format(
                        prograss='#'*int(round(10*i/num_images)) + '-'*int(round(10*(1-i/num_images))), iters=i, epoch_size=num_images,
                        time=time)
                sys.stdout.write(log)
                sys.stdout.flush()

        # write result to pkl
        with open(os.path.join(output_dir, 'detections.pkl'), 'wb') as f:
            pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)

        # currently the COCO dataset do not return the mean ap or ap 0.5:0.95 values
        print('Evaluating detections')
        data_loader.dataset.evaluate_detections(all_boxes, output_dir)

In [9]:
test_loader

In [10]:
test_epoch(model, test_loader, detector, output_dir, use_gpu)

ValueError: not enough values to unpack (expected 2, got 0)

In [19]:
state_dict['base.1.weight']

tensor([2.8344e-01, 3.5974e-01, 3.6994e-01, 3.4396e-01, 2.1134e-01, 3.5243e-01,
        2.4812e-01, 2.0826e-01, 2.0899e-01, 3.5414e-01, 2.6963e-01, 2.3821e-01,
        2.2605e-01, 2.6404e-09, 2.5536e-01, 2.0883e-01, 2.8525e-01, 2.1334e-01,
        1.9640e-01, 4.9537e-01, 1.7897e-01, 2.4970e-01, 2.1919e-01, 2.4997e-01,
        2.7225e-01, 3.1014e-01, 2.4602e-01, 2.1531e-01, 2.7553e-01, 4.5233e-01,
        2.6679e-01, 2.6546e-01, 4.4956e-01, 1.8285e-01, 4.1098e-01, 4.9113e-01,
        3.4308e-01, 2.7945e-01, 2.4083e-01, 2.3348e-01, 4.8381e-01, 2.3625e-01,
        4.6900e-01, 3.0868e-01, 5.9719e-01, 2.3564e-01, 2.7532e-01, 3.1704e-01,
        3.1421e-01, 1.9114e-01, 2.1785e-01, 2.6521e-01, 6.8765e-01, 2.3054e-01,
        2.4925e-01, 2.8236e-01, 2.3724e-01, 2.2439e-01, 5.3602e-01, 2.3448e-01,
        2.6670e-01, 2.5835e-01, 4.8405e-01, 1.7720e-01])

In [12]:
m = torch.load_("/workspace/raid/data/jgusak/ssds.pytorch/resnet50_ssd_voc_79.7.pth")

AttributeError: module 'torch' has no attribute 'load_state_dict'

In [20]:
dict(state_dict).keys()

dict_keys(['base.0.weight', 'base.1.weight', 'base.1.bias', 'base.1.running_mean', 'base.1.running_var', 'base.4.conv1.weight', 'base.4.bn1.weight', 'base.4.bn1.bias', 'base.4.bn1.running_mean', 'base.4.bn1.running_var', 'base.4.conv2.weight', 'base.4.bn2.weight', 'base.4.bn2.bias', 'base.4.bn2.running_mean', 'base.4.bn2.running_var', 'base.4.conv3.weight', 'base.4.bn3.weight', 'base.4.bn3.bias', 'base.4.bn3.running_mean', 'base.4.bn3.running_var', 'base.4.downsample.0.weight', 'base.4.downsample.1.weight', 'base.4.downsample.1.bias', 'base.4.downsample.1.running_mean', 'base.4.downsample.1.running_var', 'base.5.conv1.weight', 'base.5.bn1.weight', 'base.5.bn1.bias', 'base.5.bn1.running_mean', 'base.5.bn1.running_var', 'base.5.conv2.weight', 'base.5.bn2.weight', 'base.5.bn2.bias', 'base.5.bn2.running_mean', 'base.5.bn2.running_var', 'base.5.conv3.weight', 'base.5.bn3.weight', 'base.5.bn3.bias', 'base.5.bn3.running_mean', 'base.5.bn3.running_var', 'base.6.conv1.weight', 'base.6.bn1.weigh